In [3]:
import time
import datetime
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import tushare as ts
ts.set_token('29eaf3bcac23df4c6d025de157ab2d53beead3391fbe6e83b4ebcb6c')
pro = ts.pro_api()

import mpl_finance as mpf
import matplotlib.pyplot as plt
from matplotlib import ticker 
from matplotlib.pylab import date2num
import matplotlib as mpl
import seaborn as sns

In [4]:
from mylab.stock.myread import *
pool_df = getStockBasic(LOCAL = True ,noSTKC = True, list_data = "20180101")
print('获得上市股票总数：', len(pool_df)-1)
pool_df.head()

获得上市股票总数： 2564


,ts_code,symbol,name,area,industry,market,exchange,list_status,list_date,delist_date
0,000001.SZ,1,平安银行,深圳,银行,主板,SZSE,L,19910403,NaN
1,000002.SZ,2,万科A,深圳,全国地产,主板,SZSE,L,19910129,NaN
2,000004.SZ,4,国农科技,深圳,生物制药,主板,SZSE,L,19910114,NaN
3,000005.SZ,5,世纪星源,深圳,环境保护,主板,SZSE,L,19901210,NaN
4,000006.SZ,6,深振业A,深圳,区域地产,主板,SZSE,L,19920427,NaN


In [5]:
def readData(dir_file, stock_code, start_date = "20100101", end_date = "20200314" ):
    for file_dir , _ , files in os.walk(dir_file):
        for i,file_name in enumerate(files):
            if file_name[:9] == stock_code:
                daily_df = pd.read_csv(file_dir+file_name)
                
                break
    return daily_df

In [46]:
all_df.head()

,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,basic_eps,diluted_eps,total_revenue,revenue,...,minority_gain,oth_compr_income,t_compr_income,compr_inc_attr_p,compr_inc_attr_m_s,ebit,ebitda,insurance_exp,undist_profit,distable_profit
0,000001.SZ,20200214,20200214,20191231,1,2,1.54,1.45,1.379580e+11,1.379580e+11,...,NaN,1.528000e+09,2.972300e+10,2.972300e+10,NaN,9.581600e+10,9.915700e+10,NaN,NaN,NaN
1,000001.SZ,20191022,20191022,20190930,1,2,1.32,1.32,1.029580e+11,1.029580e+11,...,NaN,1.189000e+09,2.481000e+10,2.481000e+10,NaN,NaN,NaN,NaN,NaN,NaN
2,000001.SZ,20190808,20190808,20190630,1,2,0.85,0.78,6.782900e+10,6.782900e+10,...,NaN,7.970000e+08,1.620000e+10,1.620000e+10,NaN,4.724100e+10,4.873600e+10,NaN,NaN,NaN
3,000001.SZ,20190808,20190808,20190630,1,2,0.85,0.78,6.782900e+10,6.782900e+10,...,NaN,7.970000e+08,1.620000e+10,1.620000e+10,NaN,NaN,NaN,NaN,NaN,NaN
4,000001.SZ,20190424,20190424,20190331,1,2,0.38,0.36,3.247600e+10,3.247600e+10,...,NaN,5.990000e+08,8.045000e+09,8.045000e+09,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
def myMerge(df1,df2,on = [], how = "left" ):
    cols = [i for i in df2.columns.values if i in df1.columns.values] # in df1 and df2
    cols = [i for i in cols if i not in on ]   # not in on
    df2 = df2.drop(cols, axis = 1 )
    df =  pd.merge( df1, df2, on = on , how = how )  
    return df

In [7]:
dates = ["20151231","20161231","20171231","20181231","20191231",
         "20150930","20160930","20170930","20180930","20190930",
         "20150630","20160630","20170630","20180630","20190630",
         "20150331","20160331","20170331","20180331","20190331",
        ]
# end_date = '20180930'
for end_date in dates:
    # get income data
    dir_file = "./data/stock/finance/income/"
    all_df = pd.DataFrame()
    for i in range(len(pool_df)):
        stock_code = pool_df.loc[i,"ts_code"]
        stock_name = pool_df.loc[i,"name"]
        try:
            finance_df = readData(dir_file, stock_code)
            finance_df["end_date"] = finance_df["end_date"].astype("str")
            all_df = pd.concat([all_df,finance_df])
        except Exception as err:
            print(stock_name,err)
            pass
        if i>100:
            all_df = all_df.loc[all_df.end_date.values ==end_date,:]

    all_df = all_df.loc[all_df.end_date.values ==end_date,:]
    print(all_df.shape)
    # all_df.head()

    # get balance data
    dir_file = "./data/stock/finance/balance/"
    balance_all_df = pd.DataFrame()
    for i in range(len(pool_df)):
        stock_code = pool_df.loc[i,"ts_code"]
        stock_name = pool_df.loc[i,"name"]
        try:
            finance_df = readData(dir_file, stock_code)
            finance_df["end_date"] = finance_df["end_date"].astype("str")
            balance_all_df = pd.concat([balance_all_df,finance_df])
        except Exception as err:
            print(stock_name,err)
            pass
        if i>100:
            balance_all_df = balance_all_df.loc[balance_all_df.end_date.values ==end_date,:]

    balance_all_df = balance_all_df.loc[balance_all_df.end_date.values ==end_date,:]
    print(balance_all_df.shape)
    # balance_all_df.head()

    all_df.drop_duplicates(subset = "ts_code", inplace = True)  # drop dupe
    balance_all_df.drop_duplicates(subset = "ts_code", inplace = True)  # drop dupe
    all_df = myMerge(all_df, balance_all_df, on = ["ts_code","end_date"] , how = "inner")
    all_df.to_csv("./data/stock/finance/finance_"+end_date + ".csv", index = False)
    print(all_df.shape)
all_df.head()

(2683, 65)
(2605, 137)
(2564, 196)
(2690, 65)
(2623, 137)
(2564, 196)
(2690, 65)
(2647, 137)
(2565, 196)
(3428, 65)
(3335, 137)
(2565, 196)
(3372, 65)
(3501, 137)
(2556, 196)
(2293, 65)
(2180, 137)
(2156, 196)
(2789, 65)
(2362, 137)
(2332, 196)
(2584, 65)
(2605, 137)
(2564, 196)
(2565, 65)
(2599, 137)
(2565, 196)
(4764, 65)
(4923, 137)
(2565, 196)
(2377, 65)
(2241, 137)
(2232, 196)
(2600, 65)
(2410, 137)
(2379, 196)
(2621, 65)
(2597, 137)
(2563, 196)
(2834, 65)
(2668, 137)
(2565, 196)
(4610, 65)
(4453, 137)
(2565, 196)
(2198, 65)
(2144, 137)
(2140, 196)
(2438, 65)
(2230, 137)
(2207, 196)
(2691, 65)
(2515, 137)
(2477, 196)
(2566, 65)
(2618, 137)
(2565, 196)
(4153, 65)
(4659, 137)
(2565, 196)


,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,basic_eps,diluted_eps,total_revenue,revenue,...,lt_payroll_payable,oth_comp_income,oth_eqt_tools,oth_eqt_tools_p_shr,lending_funds,acc_receivable,st_fin_payable,payables,hfs_assets,hfs_sales
0,000001.SZ,20190424,20190424,20190331,1,2.0,0.3800,0.3600,3.247600e+10,3.247600e+10,...,NaN,1.385000e+09,2.367800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000002.SZ,20190430,20190430,20190331,1,1.0,0.1020,0.1020,4.837463e+10,4.837463e+10,...,NaN,-1.688425e+09,NaN,NaN,NaN,NaN,NaN,NaN,6.603734e+09,NaN
2,000004.SZ,20190426,20190426,20190331,1,1.0,0.0637,0.0637,1.069528e+08,1.069528e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000005.SZ,20190425,20190425,20190331,1,1.0,-0.0168,-0.0168,8.371927e+07,8.371927e+07,...,NaN,-1.100994e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000006.SZ,20190430,20190430,20190331,1,1.0,0.0821,0.0821,4.134936e+08,4.134936e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
dates = ["20150930","20160930","20170930","20180930","20190930",
         "20151231","20161231","20171231","20181231","20191231",
         "20150630","20160630","20170630","20180630","20190630"]

# total_revenue             	float	Y	营业总收入
# operate_profit            	float	Y	营业利润
# total_cogs                	float	Y	营业总成本
# n_income                  	float	Y	净利润(含少数股东损益)
# n_income_attr_p           	float	Y	净利润(不含少数股东损益)
# report_type               	str 	Y	报告类型 1合并报表 2单季合并 
# total_assets              	float	Y	资产总计
# total_hldr_eqy_exc_min_int	float	Y	股东权益合计(不含少数股东权益)
# total_hldr_eqy_inc_min_int	float	Y	股东权益合计(含少数股东权益)  净资产
try:
    del(all_df)
except:
    pass
for end_date in dates:
    df = pd.read_csv("./data/stock/finance/finance_"+end_date + ".csv")
    cols = [#"report_type","end_date_x",
            "ts_code","total_revenue","operate_profit","total_cogs","n_income","n_income_attr_p",
            "total_assets","total_hldr_eqy_inc_min_int","total_hldr_eqy_exc_min_int"]
    df = df.loc[df.report_type_x.values ==1, cols ]
    df["ROE1"] = df["n_income"]/df["total_hldr_eqy_inc_min_int"]*100
    df["ROE2"] = df["n_income_attr_p"]/df["total_hldr_eqy_exc_min_int"]*100  # 不含少数
    df["Leverage_Ratio"] = df["total_hldr_eqy_inc_min_int"]/df["total_assets"]*100  # 银行资本（所有者权益）和资产负债表中的总资产的比率
    df["income_rate"] = df["n_income"]/df["total_cogs"]*100
    df["flow_rate"] = df["total_cogs"]/df["total_hldr_eqy_inc_min_int"]*100
    cols = [c+"_"+end_date for c in df.columns.values]
    df.columns = cols
    df.rename({"ts_code"+"_"+end_date:"ts_code"},axis = 1,inplace = True)
#     df.drop(["report_type","end_date_x"],inplace = True)
    try:
        all_df = pd.merge(all_df,df, on="ts_code", how = "outer")
    except Exception as err:
        print(end_date,err)
        all_df = df
all_df.to_csv("./data/stock/finance/finance2015-2020" ".csv", index = False)
print(all_df.shape)
all_df.head()  

20150930 name 'all_df' is not defined
(3793, 196)


,ts_code,total_revenue_20150930,operate_profit_20150930,total_cogs_20150930,n_income_20150930,n_income_attr_p_20150930,total_assets_20150930,total_hldr_eqy_inc_min_int_20150930,total_hldr_eqy_exc_min_int_20150930,ROE1_20150930,...,n_income_20190630,n_income_attr_p_20190630,total_assets_20190630,total_hldr_eqy_inc_min_int_20190630,total_hldr_eqy_exc_min_int_20190630,ROE1_20190630,ROE2_20190630,Leverage_Ratio_20190630,income_rate_20190630,flow_rate_20190630
0,000001.SZ,7.115200e+10,2.330500e+10,4.784700e+10,1.774000e+10,1.774000e+10,2.599060e+12,1.571360e+11,1.571360e+11,11.289584,...,1.540300e+10,1.540300e+10,3.590766e+12,2.566030e+11,2.566030e+11,6.002658,6.002658,7.146191,32.229243,18.624880
1,000002.SZ,7.959621e+10,1.235370e+10,6.872362e+10,9.538624e+09,6.854458e+09,5.707934e+11,1.211252e+11,8.892608e+10,7.875011,...,1.928568e+10,1.184175e+10,1.577057e+12,2.324675e+11,1.610169e+11,8.296073,7.354353,14.740593,17.110413,48.485524
2,000004.SZ,8.889476e+07,9.352559e+06,8.041749e+07,7.442136e+06,2.376704e+06,3.586395e+08,1.638883e+08,8.273100e+07,4.540982,...,-4.565591e+06,-1.918825e+06,1.482165e+08,1.260689e+08,1.073170e+08,-3.621504,-1.787998,85.057284,-3.676983,98.491170
3,000005.SZ,4.575717e+07,-3.025262e+07,7.600979e+07,-2.983602e+07,-2.983602e+07,1.336589e+09,6.398790e+08,6.398790e+08,-4.662760,...,6.404029e+07,6.223482e+07,2.922820e+09,1.646294e+09,1.554414e+09,3.889966,4.003747,56.325544,19.923771,19.524246
4,000006.SZ,2.305456e+09,3.880650e+08,1.916626e+09,2.875968e+08,2.768244e+08,1.238693e+10,4.342856e+09,4.252072e+09,6.622296,...,2.124529e+08,2.034095e+08,1.556625e+10,6.463349e+09,6.137830e+09,3.287040,3.314030,41.521548,28.028011,11.727697


# rank 1

In [6]:
# ROE
all_df = pd.read_csv("./data/stock/finance/finance2015-2020" ".csv")
df = all_df.copy()
# 利润大于零
df = df.loc[df["total_revenue_20190930"].values >0, :]
df = df.loc[df["n_income_20190930"].values >0, :]
# ROE
df = df.loc[df["ROE1_20190930"].values >10, :]
df = df.loc[df["ROE1_20180930"].values >10, :]
df = df.loc[df["ROE1_20170930"].values >10, :]
# 成长
df = df.loc[df["n_income_20190930"].values >df["n_income_20180930"].values, :]
df = df.loc[df["n_income_20180930"].values >df["n_income_20170930"].values, :]
df = df.loc[df["total_revenue_20190930"].values >df["total_revenue_20180930"].values, :]
df = df.loc[df["total_revenue_20180930"].values >df["total_revenue_20170930"].values, :]
# income
df = df.loc[df["income_rate_20190630"].values >10, :]
df = df.loc[df["income_rate_20180630"].values >10, :]
df = df.loc[df["income_rate_20170630"].values >10, :]
# cash flow
df = df.loc[df["flow_rate_20190630"].values >50, :]
df = df.loc[df["flow_rate_20180630"].values >50, :]
df = df.loc[df["flow_rate_20170630"].values >50, :]
# Leverage_Ratio_20190630
df = df.loc[df["Leverage_Ratio_20190630"].values >50, :]
df = df.loc[df["Leverage_Ratio_20180630"].values >50, :]
df = df.loc[df["Leverage_Ratio_20170630"].values >50, :]

pool_df = pro.stock_basic()
daily_basic_df = pd.read_csv("./data/stock/daily_basic/daily_basic_20200324.csv")
df = pd.merge(df,pool_df,on = "ts_code",how = "left")
df = pd.merge(df,daily_basic_df,on = "ts_code",how = "left")

# pe	float	市盈率（总市值/净利润， 亏损的PE为空）
# pe_ttm	float	市盈率（TTM，亏损的PE为空）
# pb	float	市净率（总市值/净资产）
# ps	float	市销率
# ps_ttm	float	市销率（TTM）
# dv_ratio	float	股息率 （%）

df = df.loc[df["pb"].values < 10, :]
df = df.loc[df["pe_ttm"].values < 50, :]
df = df.loc[df["dv_ratio"].values < 4, :]
df.to_csv("./data/stock/blue_stock_rank_1.csv", index = False)

print(df.shape)
print(df.name.values)
df.head()

(12, 220)
['三花智控' '大华股份' '恩华药业' '海康威视' '生益科技' '中国国旅' '桃李面包' '利安隆' '绝味食品' '顾家家居'
 '元隆雅图' '欧派家居']


,ts_code,total_revenue_20150930,operate_profit_20150930,total_cogs_20150930,n_income_20150930,n_income_attr_p_20150930,total_assets_20150930,total_hldr_eqy_inc_min_int_20150930,total_hldr_eqy_exc_min_int_20150930,ROE1_20150930,...,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,002050.SZ,4.797484e+09,5.734089e+08,4.225289e+09,5.149705e+08,5.096419e+08,7.698063e+09,4.186731e+09,4.149717e+09,12.300062,...,4.9002,4.0577,3.8830,1.2035,1.2035,276536.6698,240663.6113,127434.6931,4.396933e+06,3.826551e+06
1,002236.SZ,5.830194e+09,6.419953e+08,5.197527e+09,7.601419e+08,7.611857e+08,8.893382e+09,5.873845e+09,5.864558e+09,12.941128,...,3.2114,2.1048,1.9092,0.6018,0.6018,300799.1230,181364.0519,153458.9676,4.981233e+06,3.003389e+06
3,002262.SZ,2.025174e+09,2.529549e+08,1.772219e+09,2.136901e+08,2.158849e+08,2.609461e+09,1.785728e+09,1.772166e+09,11.966554,...,2.9585,2.7743,2.5796,0.5715,0.5715,101938.7711,88722.9194,49150.2459,1.070357e+06,9.315907e+05
4,002415.SZ,1.667764e+10,3.756159e+09,1.299901e+10,3.782900e+09,3.769987e+09,2.545422e+10,1.713614e+10,1.709068e+10,22.075564,...,5.7981,5.2203,4.5049,2.1560,2.1560,934501.0696,809027.2351,328398.3907,2.601651e+07,2.252332e+07
6,600183.SH,5.612947e+09,3.854247e+08,5.194217e+09,3.424266e+08,3.381407e+08,8.782301e+09,4.569075e+09,4.445727e+09,7.494441,...,7.0778,5.2245,4.7274,1.1859,1.1859,227619.1340,227619.1340,109928.2839,6.259526e+06,6.259526e+06


# rank 2

In [17]:
# ROE
all_df = pd.read_csv("./data/stock/finance/finance2015-2020" ".csv")
df = all_df.copy()
# 利润大于零
df = df.loc[df["total_revenue_20190930"].values >0, :]
df = df.loc[df["n_income_20190930"].values >0, :]
# ROE
df = df.loc[df["ROE1_20190930"].values >10, :]
df = df.loc[df["ROE1_20180930"].values >10, :]
df = df.loc[df["ROE1_20170930"].values >10, :]
# 成长
df = df.loc[df["n_income_20190930"].values >df["n_income_20180930"].values, :]
df = df.loc[df["n_income_20180930"].values >df["n_income_20170930"].values, :]
df = df.loc[df["total_revenue_20190930"].values >df["total_revenue_20180930"].values, :]
df = df.loc[df["total_revenue_20180930"].values >df["total_revenue_20170930"].values, :]
# income
df = df.loc[df["income_rate_20190630"].values > 0, :]
df = df.loc[df["income_rate_20180630"].values > 0, :]
df = df.loc[df["income_rate_20170630"].values > 0, :]

pool_df = pro.stock_basic()
daily_basic_df = pd.read_csv("./data/stock/daily_basic/daily_basic_20200327.csv")
df = pd.merge(df,pool_df,on = "ts_code",how = "left")
df = pd.merge(df,daily_basic_df,on = "ts_code",how = "left")

# pe	float	市盈率（总市值/净利润， 亏损的PE为空）
# pe_ttm	float	市盈率（TTM，亏损的PE为空）
# pb	float	市净率（总市值/净资产）
# ps	float	市销率
# ps_ttm	float	市销率（TTM）
# dv_ratio	float	股息率 （%）

df = df.loc[df["pb"].values < 10, :]
df = df.loc[df["pe_ttm"].values < 40, :]
df = df.loc[df["dv_ratio"].values < 7, :]
df = df.loc[df["close"].values < 20, :]
df.to_csv("./data/stock/blue_stock_rank_2.csv", index = False)

print(df.shape)
print(df.name.values)
df.head()

(64, 220)
['潍柴动力' '承德露露' '华东医药' '三花智控' '金螳螂' '东港股份' '露天煤业' '荣盛发展' '东华能源' '大华股份'
 '恩华药业' '奇正藏药' '伟星新材' '和而泰' '嘉事堂' '森马服饰' '索菲亚' '豪迈科技' '龙蟒佰利' '浙江美大' '良信电器'
 '国光股份' '北陆药业' '青松股份' '洲明科技' '安科瑞' '花园生物' '九强生物' '乐凯新材' '濮阳惠成' '名家汇' '伊力特'
 '圆通速递' '西藏天路' '兰太实业' '恒力石化' '浙江龙盛' '龙净环保' '通威股份' '涪陵电力' '尖峰集团' '海尔智家'
 '苏美达' '茂业商业' '航民股份' '宝丰能源' '文山电力' '南京银行' '兴业银行' '建设银行' '迎驾贡酒' '富森美'
 '长沙银行' '贵阳银行' '杭叉集团' '苏利股份' '元祖股份' '周大生' '金龙羽' '佛燃能源' '成都银行' '银都股份' '风语筑'
 '南都物业']


,ts_code,total_revenue_20150930,operate_profit_20150930,total_cogs_20150930,n_income_20150930,n_income_attr_p_20150930,total_assets_20150930,total_hldr_eqy_inc_min_int_20150930,total_hldr_eqy_exc_min_int_20150930,ROE1_20150930,...,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
2,000338.SZ,5.457171e+10,2.066102e+09,5.261066e+10,1.565796e+09,1.042219e+09,1.197740e+11,4.116953e+10,3.143660e+10,3.803288,...,2.2140,0.5742,0.5742,3.4073,3.4073,793387.3895,424500.1625,417013.0701,1.001255e+07,5.357192e+06
9,000848.SZ,2.131644e+09,4.691655e+08,1.662068e+09,3.486719e+08,3.443915e+08,2.005608e+09,1.581412e+09,1.557532e+09,22.048135,...,3.4418,3.0713,2.9351,6.0060,6.0060,97856.2728,97835.5383,58023.5505,6.517228e+05,6.515847e+05
11,000963.SZ,1.615636e+10,1.163304e+09,1.499723e+10,9.727531e+08,9.274152e+08,1.134699e+10,3.058835e+09,2.801895e+09,31.801424,...,2.6048,0.9907,0.8658,1.5841,1.5841,174980.9548,174978.4303,73084.6146,3.037669e+06,3.037626e+06
14,002050.SZ,4.797484e+09,5.734089e+08,4.225289e+09,5.149705e+08,5.096419e+08,7.698063e+09,4.186731e+09,4.149717e+09,12.300062,...,4.9650,4.1113,3.9342,1.1878,1.1878,276536.6698,240663.6113,127434.6931,4.455006e+06,3.877091e+06
15,002081.SZ,1.458847e+10,1.579654e+09,1.310529e+10,1.324879e+09,1.335108e+09,2.423620e+10,8.475514e+09,8.355094e+09,15.631846,...,1.5212,0.8827,0.7178,2.4170,2.4170,268440.8689,259621.4928,130836.7334,2.214637e+06,2.141877e+06


# rank 3

In [20]:
# ROE
all_df = pd.read_csv("./data/stock/finance/finance2015-2020" ".csv")
df = all_df.copy()
# 利润大于零
df = df.loc[df["total_revenue_20190930"].values >0, :]
df = df.loc[df["total_revenue_20181231"].values >0, :]
df = df.loc[df["total_revenue_20190930"].values >0, :]
df = df.loc[df["total_revenue_20181231"].values >0, :]
# 成长
df = df.loc[df["n_income_20190930"].values >df["n_income_20180930"].values, :]
df = df.loc[df["n_income_20180930"].values >df["n_income_20170930"].values, :]
df = df.loc[df["total_revenue_20190930"].values >df["total_revenue_20180930"].values, :]
df = df.loc[df["total_revenue_20180930"].values >df["total_revenue_20170930"].values, :]
# income
df = df.loc[df["income_rate_20190630"].values > 0, :]
df = df.loc[df["income_rate_20180630"].values > 0, :]
df = df.loc[df["income_rate_20170630"].values > 0, :]

pool_df = pro.stock_basic()
daily_basic_df = pd.read_csv("./data/stock/daily_basic/daily_basic_20200327.csv")
df = pd.merge(df,pool_df,on = "ts_code",how = "left")
df = pd.merge(df,daily_basic_df,on = "ts_code",how = "left")

# pe	float	市盈率（总市值/净利润， 亏损的PE为空）
# pe_ttm	float	市盈率（TTM，亏损的PE为空）
# pb	float	市净率（总市值/净资产）
# ps	float	市销率
# ps_ttm	float	市销率（TTM）
# dv_ratio	float	股息率 （%）
df = df.loc[df["pb"].values < 10, :]
df = df.loc[df["pe_ttm"].values < 40, :]
df = df.loc[df["dv_ratio"].values < 7, :]
df = df.loc[df["close"].values < 15, :]
df = df.loc[df["market"].values != "创业板", :]
df.to_csv("./data/stock/blue_stock_rank_3.csv", index = False)

print(df.shape)
print(df.name.values)
df.head()

(353, 220)
['平安银行' '飞亚达' '华侨城A' '海王生物' '中信海直' '华数传媒' '中联重科' '东方盛虹' '潍柴动力' '英特集团'
 '徐工机械' '珠海港' '岭南控股' '柳工' '广弘控股' '粤电力A' '我爱我家' '海德股份' '北部湾港' '兴蓉环境' '铜陵有色'
 '仁和药业' '金科股份' '阳光城' '恒逸石化' '浙江震元' '中兴商业' '中原传媒' '泰禾集团' '新华制药' '中核科技'
 '创维数字' '承德露露' '华茂股份' '国风塑业' '天山股份' '赣能股份' '现代投资' '浙商中拓' '大亚圣象' '新乡化纤'
 '中南建设' '盈峰环境' '蓝焰控股' '西山煤电' '中航机电' '永新股份' '京新药业' '苏宁易购' '旺能环境' '粤水电'
 '宏润建设' '中材科技' '金螳螂' '鲁阳节能' '山河智能' '海翔药业' '莱宝高科' '威海广泰' '中国海诚' '东港股份'
 '露天煤业' '东南网架' '拓邦股份' '东华科技' '荣盛发展' '通润装备' '云海金属' '广百股份' '劲嘉股份' '海亮股份'
 '东华能源' '濮耐股份' '通产丽星' '北化股份' '泰和新材' '恩华药业' '华昌化工' '天润曲轴' '太阳电缆' '美盈森'
 '众生药业' '久立特材' '皖通科技' '仙琚制药' '伟星新材' '双箭股份' '长青股份' '北京利尔' '垒知集团' '中远海科'
 '和而泰' '爱仕达' '嘉欣丝绸' '远东传动' '江苏神通' '闰土股份' '众业达' '金洲管道' '巨星科技' '珠江啤酒' '常宝股份'
 '江海股份' '大金重工' '日发精机' '天顺风能' '金杯电工' '司尔特' '云图控股' '中化岩土' '东方铁塔' '新联电子'
 '千红制药' '森马服饰' '中京电子' '好想你' '山东章鼓' '龙蟒佰利' '世纪华通' '瑞和股份' '道明光学' '永高股份'
 '万润股份' '华宏科技' '博彦科技' '利君股份' '德联集团' '浙江美大' '乔治白' '金河生物' '红旗连锁' '博实股份'
 '良信电器' '海洋王' '富煌钢构' '木林森' '仙坛股份' '国光股份' '鹭燕医药' '浦发银行' '中国国贸' '首创股

,ts_code,total_revenue_20150930,operate_profit_20150930,total_cogs_20150930,n_income_20150930,n_income_attr_p_20150930,total_assets_20150930,total_hldr_eqy_inc_min_int_20150930,total_hldr_eqy_exc_min_int_20150930,ROE1_20150930,...,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,000001.SZ,7.115200e+10,2.330500e+10,4.784700e+10,1.774000e+10,1.774000e+10,2.599060e+12,1.571360e+11,1.571360e+11,11.289584,...,0.9346,1.8498,1.8498,0.9756,0.9756,1.940592e+06,1.940575e+06,860111.1956,2.551878e+07,2.551856e+07
3,000026.SZ,2.467881e+09,1.352111e+08,2.333110e+09,1.070606e+08,1.066149e+08,3.852330e+09,1.708490e+09,1.704829e+09,6.266387,...,1.4785,1.0595,1.0595,2.2358,2.2358,4.429689e+04,3.566923e+04,19371.4963,3.924704e+05,3.160294e+05
9,000069.SZ,1.743661e+10,3.859597e+09,1.397188e+10,2.860959e+09,2.491276e+09,1.031446e+11,3.389503e+10,2.997892e+10,8.440645,...,0.8593,1.1126,1.0020,4.5951,4.5951,8.202506e+05,7.035147e+05,420014.0017,5.356237e+06,4.593951e+06
10,000078.SZ,8.221318e+09,4.337172e+08,8.094439e+09,3.932218e+08,3.507192e+08,1.216369e+10,2.170980e+09,1.829979e+09,18.112644,...,2.0881,0.3469,0.3227,0.4149,0.4149,2.762583e+05,2.621961e+05,140551.5680,1.331565e+06,1.263785e+06
11,000099.SZ,9.120046e+08,1.642831e+08,7.492174e+08,1.465861e+08,1.442167e+08,4.712436e+09,2.824300e+09,2.792370e+09,5.190176,...,1.1565,2.4131,2.4131,0.4006,0.4006,6.060704e+04,6.060704e+04,37195.0946,3.781879e+05,3.781879e+05


# rank 4

In [21]:
# ROE
all_df = pd.read_csv("./data/stock/finance/finance2015-2020" ".csv")
df = all_df.copy()
# 利润大于零
df = df.loc[df["total_revenue_20190930"].values >0, :]
df = df.loc[df["total_revenue_20181231"].values >0, :]
df = df.loc[df["total_revenue_20190930"].values >0, :]
df = df.loc[df["total_revenue_20181231"].values >0, :]
# income
df = df.loc[df["income_rate_20190630"].values > 0, :]
df = df.loc[df["income_rate_20180630"].values > 0, :]
df = df.loc[df["income_rate_20170630"].values > 0, :]

pool_df = pro.stock_basic()
daily_basic_df = pd.read_csv("./data/stock/daily_basic/daily_basic_20200327.csv")
df = pd.merge(df,pool_df,on = "ts_code",how = "left")
df = pd.merge(df,daily_basic_df,on = "ts_code",how = "left")

# pe	float	市盈率（总市值/净利润， 亏损的PE为空）
# pe_ttm	float	市盈率（TTM，亏损的PE为空）
# pb	float	市净率（总市值/净资产）
# ps	float	市销率
# ps_ttm	float	市销率（TTM）
# dv_ratio	float	股息率 （%）
df = df.loc[df["pb"].values < 10, :]
df = df.loc[df["pe_ttm"].values < 40, :]
df = df.loc[df["dv_ratio"].values < 7, :]
df = df.loc[df["close"].values < 15, :]
df = df.loc[df["market"].values != "创业板", :]
df.to_csv("./data/stock/blue_stock_rank_4.csv", index = False)

print(df.shape)
# print(df.name.values)
df.head()

(955, 220)
['平安银行' '深振业A' '神州高铁' '深物业A' '南玻A' '飞亚达' '深圳能源' '富奥股份' '大悦城' '中集集团' '中洲控股'
 '泛海控股' '深天马A' '方大集团' '华锦股份' '中金岭南' '深圳华强' '北方国际' '中国长城' '华侨城A' '特发信息'
 '海王生物' '盐田港' '深圳机场' '天健集团' '中信海直' 'TCL科技' '丰原药业' '华数传媒' '中联重科' '申万宏源'
 '东方盛虹' '潍柴动力' '金融街' '长虹华意' '英特集团' '东旭光电' '合肥百货' '通程控股' '南京公用' '徐工机械'
 '晨鸣纸业' '山东路桥' '鄂武商A' '珠海港' '岭南控股' '红太阳' '柳工' '广弘控股' '冰山冷热' '穗恒运A' '广宇发展'
 '粤电力A' '佛山照明' '中原环保' '金圆股份' '湖南投资' '靖远煤电' '安道麦A' '万向钱潮' '我爱我家' '陕国投A'
 '渝三峡A' '海德股份' '北部湾港' '太阳能' '兴蓉环境' '建投能源' '韶能股份' '渤海股份' '华媒控股' '中油资本'
 '新华联' '天茂集团' '铜陵有色' '顺发恒业' '茂化实华' '西王食品' '仁和药业' '金科股份' '经纬纺机' '阳光城'
 '智度股份' '中山公用' '东北证券' '宝新能源' '恒逸石化' '浙江震元' '河钢股份' '中兴商业' '中原传媒' '湖南发展'
 '鲁泰A' '国元证券' '四川美丰' '泰禾集团' '漳州发展' '新华制药' '本钢板材' '广发证券' '中核科技' '新兴铸管'
 '长江证券' '万年青' '中国武夷' '创维数字' '冰轮环境' '山东海化' '超声电子' '太钢不锈' '东莞控股' '鲁西化工'
 '承德露露' '华茂股份' '国风塑业' '天山股份' '中广核技' '湖北能源' '城发环境' '中鼎股份' '峨眉山A' '中嘉博创'
 '鞍钢股份' '赣能股份' '现代投资' '新洋丰' '浙商中拓' '大亚圣象' '电广传媒' '金陵药业' '南方汇通' '海信家电'
 '福星股份' '中钢国际' '中粮科技' '神火股份' '华西股份' '冀中能源' '新乡化纤' '中南建设' '盈峰环境' '蓝

,ts_code,total_revenue_20150930,operate_profit_20150930,total_cogs_20150930,n_income_20150930,n_income_attr_p_20150930,total_assets_20150930,total_hldr_eqy_inc_min_int_20150930,total_hldr_eqy_exc_min_int_20150930,ROE1_20150930,...,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,000001.SZ,7.115200e+10,2.330500e+10,4.784700e+10,1.774000e+10,1.774000e+10,2.599060e+12,1.571360e+11,1.571360e+11,11.289584,...,0.9346,1.8498,1.8498,0.9756,0.9756,1.940592e+06,1.940575e+06,860111.1956,2.551878e+07,2.551856e+07
2,000006.SZ,2.305456e+09,3.880650e+08,1.916626e+09,2.875968e+08,2.768244e+08,1.238693e+10,4.342856e+09,4.252072e+09,6.622296,...,0.9172,2.4615,1.6588,4.2576,4.2576,1.349995e+05,1.348308e+05,86230.9824,6.182977e+05,6.175251e+05
3,000008.SZ,7.631092e+08,1.108212e+08,6.645145e+08,1.027108e+08,9.581449e+07,3.359762e+09,2.817054e+09,2.817054e+09,3.646038,...,1.2580,3.5344,3.1972,0.3047,0.3047,2.780795e+05,2.665297e+05,175163.1419,9.065393e+05,8.688870e+05
5,000011.SZ,6.360503e+08,6.377790e+07,5.796274e+08,4.645580e+07,4.645580e+07,4.008625e+09,1.985561e+09,1.984699e+09,2.339681,...,1.5714,1.2486,1.2486,3.6145,3.6145,5.959791e+04,5.264426e+04,14606.0346,4.946626e+05,4.369473e+05
6,000012.SZ,5.377130e+09,3.401294e+08,5.178089e+09,4.089526e+08,3.940694e+08,1.523499e+10,7.710081e+09,7.707197e+09,5.304128,...,1.5042,1.3534,1.4189,0.9839,0.9839,3.108196e+05,1.957550e+05,187091.6570,1.435987e+06,9.043881e+05
